In [1]:
## importing 'mysql.connector' 
import mysql.connector 
import config
from mysql.connector import errorcode
import json
import pandas as pd
import requests

In [3]:
## Connect to DB server on AWS
## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.admin,
    passwd = config.password,
   # database = "Movies"       #can say what database we expect everything loaded into
)

In [4]:
cursor = cnx.cursor()
db_name = "Yelp"

In [6]:
#create new db if it doesn't exist
## Create new DB 
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

create_database(cursor, db_name)       #when we create a database we need to declare a name/where it will be stored

Failed creating database: 1007 (HY000): Can't create database 'Movies'; database exists


In [54]:
create_query = """
CREATE TABLE rest (
      ID varchar(30) NOT NULL,
      name varchar(50),
      url varchar(100),
      review_count int(8),
      rating float(5),
      price varchar(4)
    )
"""

In [55]:
#uncomment if I need to drop table
#cursor.execute("DROP Table reviews;")

In [56]:
x = 'i am x'

In [57]:
f'hey whats up {x} now were done' 


'hey whats up i am x now were done'

In [58]:
#try to create database table
try:
    print("Creating a new table")
    cursor.execute(create_query)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")
        
cursor.close()
cnx.close()

Creating a new table
already exists.


In [59]:
create_review_query = """
CREATE TABLE reviews (
      rev_ID varchar(30) NOT NULL,
      rest_ID varchar(30) NOT NULL,
      
    )
"""###NEED MORE STUFF HERE

In [60]:
## Connect to DB server on AWS
## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.admin,
    passwd = config.password,
    database = "Yelp")

cursor = cnx.cursor()
db_name = "Yelp"

try:
    print("Creating reviews table")
    cursor.execute(create_review_query)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")
        
cursor.close()
cnx.close()

Creating reviews table
You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ')' at line 5


In [61]:
##Special DELETE table values leave commented out unless we want to truncate
cnx = mysql.connector.connect(
    host = config.host,
    user = config.admin,
    passwd = config.password,
    database = "Yelp")

cursor = cnx.cursor()
db_name = "Yelp"

cursor.execute("TRUNCATE Table rest;")

cursor.close()
cnx.close()

In [62]:
# Create a table for the rest
api_key = "pVQ1_kO1_bCk9EOvKaQGSoVd3ck5oYUbeTICdg4boxz5BEFKRfeUj6kIFlOmtljM81-UT6yUFLSaK8fJ01SfsBWuIda0n5jbfw0nFVI9lJQQkIUM32VGo3L2VtuxXXYx"


In [63]:
#define fxn for use later
#this function takes in a list of tuples and insert them into the database
def db_insert(parsed):
    #open connection
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.admin,
    passwd = config.password,
    database = "Yelp")
    
    #create cursor
    cursor = cnx.cursor()
    
    
    insert_statement = ("""INSERT INTO rest 
                    (ID, name, url, review_count, rating, price) 
                    VALUES (%s,%s,%s,%s,%s,%s)""")
    
    cursor.executemany(insert_statement, parsed)
    cnx.commit() 
    
    cursor.close()
    cnx.close()

In [64]:
def rating_to_num(rate):
    if type(rate)==str:
        return len(rate)
    else:
        return 0

In [65]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB
def parsed_results(business_dict):
    df = pd.DataFrame(business_dict)
    df = df[['id', 'name', 'url', 'review_count', 'rating', 'price']]
    df['price'] = df['price'].apply(rating_to_num)
    list_of_tuples = list(df.itertuples(index=False,name=None))
    return list_of_tuples

In [66]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [67]:
import time

def all_results(url_params, api_key):
    num = response.json()['total']        #returns total of last call
    print('{} total matches found.'.format(num))
    cur = 50
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        data = parsed_results(results)
        db_insert(data)
        time.sleep(1) #Wait a second
        cur += 50
    print("All uploaded")

In [68]:
url_params['term']

'Veg+et+ari+an+Food'

In [69]:
term = 'Vegetarian Food'
location = 'Financial District, Manhattan, NY'
SEARCH_LIMIT = 50

url = 'https://api.yelp.com/v3/businesses/search'
cur =0

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': cur
            }
#initial yelp call
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)

first = response.json()['businesses']


#initial parse

data = parsed_results(first)

#initial storage
db_insert(data)

#initial wait
time.sleep(1) #Wait a second
#cur starts at 50 since we already did one database commit
cur += 50

In [70]:
#Now finally we can call all_results
all_results(url_params, api_key)


1500 total matches found.
All uploaded


In [21]:
#get IDs for SQL to look up review tables
#open connection
cnx = mysql.connector.connect(
host = config.host,
user = config.admin,
passwd = config.password,
database = "Yelp")

#create cursor
cursor = cnx.cursor()

cursor.execute('SELECT DISTINCT ID FROM rest;')

ids = cursor.fetchall()
               
    

In [41]:
ids[0]

('FvCslLyo0SlDKI52bEDekQ',)

In [34]:
# url_params = {
#                 'term': term.replace(' ', '+'),
#                 'location': location.replace(' ', '+'),
#                 'limit': SEARCH_LIMIT,
#                 'offset': cur
#             }
#initial yelp call
biz = 'FvCslLyo0SlDKI52bEDekQ'
url = "https://api.yelp.com/v3/businesses/{}/reviews".format(biz)
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers) #, params=url_params)

first_rev = response.json()#['reviews']



In [43]:
def yelp_call_review(url_params, api_key, biz_id):
    rev_keys = ['id', 'text', 'rating']    
    url = "https://api.yelp.com/v3/businesses/{}/reviews".format(biz_id)
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()['reviews']
    list_of_rev = []
    for review in data:
        temp_tuple = ()
        if rev_keys in review.keys():
            temp_tuple = (review['id'], review['text'], review['rating'])
        else:
            temp_tuple = ('','','')
        list_of_rev.append(temp_tuple)
             
    return list_of_rev
    

In [ ]:
db_list = yelp_call_review(url_params, api_key, biz_id)

cnx = mysql.connector.connect(
host = config.host,
user = config.admin,
passwd = config.password,
database = "Yelp")

#create cursor
cursor = cnx.cursor()


insert_statement = ("""INSERT INTO review 
                (ID, name, url, review_count, rating, price) 
                VALUES (%s,%s,%s,%s,%s,%s)""")

cursor.executemany(insert_statement, parsed)
cnx.commit() 

cursor.close()
cnx.close()

In [35]:
first_rev.keys()

dict_keys(['reviews', 'total', 'possible_languages'])

In [39]:
(first_rev['reviews'][0]).keys()

dict_keys(['id', 'url', 'text', 'rating', 'time_created', 'user'])

In [ ]:
# df = pd.DataFrame(results)
# df = df[['id', 'name', 'url', 'review_count', 'rating', 'price']]
# df

In [ ]:
# df['price'].dtype

In [ ]:
# def rating_to_num(rate):
#     if type(rate)==str:
#         return len(rate)
#     else:
#         return 0
            

In [ ]:
# df['price'] = df['price'].apply(rating_to_num)
# df

In [ ]:
# df['price'].fillna('', inplace=True)

# df